## Importing the Libraries

In [1]:
import pandas as pd
import numpy as np
from math import floor

import matplotlib.pyplot as plt

import os
import itertools

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, matthews_corrcoef
from sklearn.preprocessing import LabelBinarizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, ReLU, Input
from tensorflow.keras.optimizers import Adam

import torch
import torch.nn as nn
import torch.nn.functional as F

import psutil
import time
import logging
import shutil

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.air import session
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.search.optuna import OptunaSearch

import helper

# Initialize Ray
ray.init(ignore_reinit_error=True)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#tf.debugging.set_log_device_placement(True)

# Check if MPS (Apple Silicon) or CUDA (Nvidia) GPU is available
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Using device: {device}")

2024-08-08 10:34:55,778	INFO worker.py:1772 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Num GPUs Available:  1
Using device: mps


## Existing Data Files Check 

In [2]:
def clear_folder(folder_path):
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if item == '.gitignore':
            continue
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)

clear_folder("ray_results")
clear_folder("outputs")
clear_folder("results")

## Feature Based MTL

In [3]:
# TODO activation for now only relu
class FeatureBasedMTLModel(nn.Module):
    def __init__(self, hidden_layers_shared=[50,30], hidden_layers_outputs=[[10],[10],[10],[10],[10]], activation='relu', dropout_rate=0.3):
        super(FeatureBasedMTLModel, self).__init__()

        self.activation = activation

        # shared LSTM layers
        self.lstm_layers = nn.ModuleList()
        self.lstm_layers.append(nn.LSTM(input_size=1, hidden_size=32, batch_first=True))
        self.lstm_layers.append(nn.LSTM(input_size=32, hidden_size=64, batch_first=True))

        input_dim = 64  # Adjusted input dimension after LSTM layers

        self.shared_layers = nn.ModuleList()
        for hidden_layer in hidden_layers_shared:
            self.shared_layers.append(nn.Linear(input_dim, hidden_layer))
            self.shared_layers.append(nn.Dropout(dropout_rate))
            input_dim = hidden_layer

        # output layers
        self.output_a2 = self.create_output_layers(hidden_layers_outputs[0], input_dim, 2)
        self.output_a3 = self.create_output_layers(hidden_layers_outputs[1], input_dim, 2)
        self.output_a4 = self.create_output_layers(hidden_layers_outputs[2], input_dim, 3)
        self.output_a12 = self.create_output_layers(hidden_layers_outputs[3], input_dim, 6)
        self.output_a21 = self.create_output_layers(hidden_layers_outputs[4], input_dim, 2)

    def create_output_layers(self, hidden_layers, input_dim, output_dim):
        layers = nn.ModuleList()
        for hidden_layer in hidden_layers:
            layers.append(nn.Linear(input_dim, hidden_layer))
            input_dim = hidden_layer
        layers.append(nn.Linear(input_dim, output_dim))
        return layers

    def forward(self, x: torch.Tensor, task_id: str):
        # Add input size dimension
        x = x.unsqueeze(-1)  # Shape: (batch_size, sequence_length, input_size)
        
        # LSTM layers
        for lstm in self.lstm_layers:
            x, (hn, cn) = lstm(x)

        x = x[:, -1, :]  # Take the output of the last LSTM cell

        # shared layers
        for layer in self.shared_layers:
            if isinstance(layer, nn.Linear):
                x = layer(x)
                x = self.apply_activation(x)
            elif isinstance(layer, nn.Dropout):
                x = layer(x)
        
        # output layers
        if task_id == 'a2':
            return self.forward_output_layers(x, self.output_a2)
        elif task_id == 'a3':
            return self.forward_output_layers(x, self.output_a3)
        elif task_id == 'a4':
            return self.forward_output_layers(x, self.output_a4)
        elif task_id == 'a12':
            return self.forward_output_layers(x, self.output_a12)
        elif task_id == 'a21':
            return self.forward_output_layers(x, self.output_a21)
        else:
            raise ValueError(f'Invalid task_id: {task_id}')
        

    def forward_output_layers(self, x, layers):
        for layer in layers[:-1]:
            x = layer(x)
            x = self.apply_activation(x)
            
        x = layers[-1](x)
        return F.softmax(x, dim=1)
    
    def apply_activation(self, x):
        if self.activation == 'relu':
            return F.relu(x)
        elif self.activation == 'tanh':
            return F.tanh(x)
        elif self.activation == 'sigmoid':
            return F.sigmoid(x)
        else:
            raise ValueError(f'Invalid activation: {self.activation}')

In [4]:

def create_batches(X_train, y_train, batch_size):
    X_train_batches = {'a2': {}, 'a3': {}, 'a4': {}, 'a12': {}, 'a21': {}}
    y_train_batches = {'a2': {}, 'a3': {}, 'a4': {}, 'a12': {}, 'a21': {}}

    for dataset in helper.dataset_list:

        for i in range(0, floor(len(X_train[dataset])/batch_size)):
                X_train_batches[dataset][str(i)] =  X_train[dataset].iloc[i* batch_size:i*batch_size+batch_size]
                y_train_batches[dataset][str(i)] =  y_train[dataset].iloc[i* batch_size:i* batch_size+batch_size]

    return X_train_batches, y_train_batches

# batchsize in this case refers to the size of batch per dataset
def train(model, X_train, y_train, num_epochs, batch_size, learning_rate):

        model = model.to(device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        max_number_of_batches = max(len(X_train[dataset]) // batch_size for dataset in helper.dataset_list)

        X_train, y_train = create_batches(X_train, y_train, batch_size)

        for epoch in range(num_epochs):
                optimizer.zero_grad()

                losses_per_epoch = {'a2': 0, 'a3': 0, 'a4': 0, 'a12': 0, 'a21': 0}

                for batch_number in range(0, max_number_of_batches):

                        for dataset in helper.dataset_list:

                                if len(X_train[dataset]) > batch_number:

                                        X_train_tensor = torch.tensor(X_train[dataset][str(batch_number)].values, dtype=torch.float32, device=device)
                                        y_train_tensor= torch.tensor(y_train[dataset][str(batch_number)].values, dtype=torch.float32, device=device)
                                        outputs = model(X_train_tensor, task_id = dataset)
                                        loss = criterion(outputs, y_train_tensor)

                                        losses_per_epoch[dataset] += loss

                for dataset in helper.dataset_list:
                        losses_per_epoch[dataset] /= len(X_train[dataset])

                # for now sum of average loss per dataset
                loss = sum(losses_per_epoch.values())
                loss.backward()
                optimizer.step()

        return model


def mcc(fn, fp, tn, tp):
    return (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

def evaluate(model, X_test, y_test, device='cpu'):

    eval_dict = {'a2': {}, 'a3': {}, 'a4': {}, 'a12': {}, 'a21': {}}
    num_classes = {'a2': 2, 'a3': 2, 'a4': 3, 'a12': 6, 'a21': 2}

    model = model.to(device)

    for dataset in helper.dataset_list:
        X_test_tensor = torch.tensor(X_test[dataset].values, dtype=torch.float32, device=device)
        # y_test_tensor = torch.tensor(y_test[dataset].values, dtype=torch.float32)
        y_pred_tensor = model(X_test_tensor, task_id=dataset)
        y_pred = y_pred_tensor.detach().cpu().numpy()  # Move tensor to CPU before converting to NumPy
        y_pred = np.argmax(y_pred, axis=1)

        # print(y_test[dataset])
        y_test_np = y_test[dataset].to_numpy()
        # print(y_test_np)
        y_test_np = np.argmax(y_test_np, axis=1)
        # print(y_test_np)

        # Calculate classification metrics using one-hot encoded targets
        eval_dict[dataset]['accuracy'] = accuracy_score(y_test_np, y_pred)
        eval_dict[dataset]['micro_f1'] = f1_score(y_test_np, y_pred, average='micro')
        eval_dict[dataset]['macro_f1'] = f1_score(y_test_np, y_pred, average='macro')
        # gets 0 quite often...???
        eval_dict[dataset]['mcc'] = matthews_corrcoef(y_test_np, y_pred)

    return eval_dict

In [5]:
def evaluate_model_on_dataset_one_split(split_index, config):

    X_train, X_test, y_train, y_test =  helper.get_joined_train_test_folds(split_index)
    # print(y_test)

    model = FeatureBasedMTLModel(activation=config['activation'], hidden_layers_shared=config['shared_hidden_layers'], hidden_layers_outputs=[
        config['a2_output_hidden_layers'], config['a3_output_hidden_layers'], config['a4_output_hidden_layers'], config['a12_output_hidden_layers'], config['a21_output_hidden_layers'],
        config['dropout_rate']
    ])

    # Train the model and collect performance data
    model = train(model, X_train, y_train, num_epochs=config['epochs'], batch_size=config['batch_size'], learning_rate=config['learning_rate'],)
    # Evaluate the model and collect performance data
    eval_dict = evaluate(model, X_test, y_test)

    return eval_dict

In [6]:
df_data_spike_1_split = {
    'a2': pd.DataFrame(),
    'a3': pd.DataFrame(),
    'a4': pd.DataFrame(),
    'a12': pd.DataFrame(),
    'a21': pd.DataFrame()
}

df_data_spike_full_split = pd.DataFrame()

data_spike_exec_1_split_dict = dict()
data_spike_exec_full_split_dict = {
    'a2': {},
    'a3': {},
    'a4': {},
    'a12': {},
    'a21': {}
}


best_params_list_getting = []

def custom_trial_dirname(trial):
    return f"trial_{trial.trial_id}"

In [7]:
def train_and_evaluate(config):
    
    global data_spike_exec_1_split_dict
    global data_spike_exec_full_split_dict
    global df_data_spike_1_split
    global results_dir

    overall_result = {
    "a2": {
        "accuracy_scores": [],
        "f1_macro_scores": [],
        "f1_micro_scores": [],
        "mcc_scores": []
    },
    "a3": {
        "accuracy_scores": [],
        "f1_macro_scores": [],
        "f1_micro_scores": [],
        "mcc_scores": []
    },
    "a4": { 
        "accuracy_scores": [],
        "f1_macro_scores": [],
        "f1_micro_scores": [],
        "mcc_scores": []
    },
    "a12": {
        "accuracy_scores": [],
        "f1_macro_scores": [],
        "f1_micro_scores": [],
        "mcc_scores": []
    },
    "a21": {
        "accuracy_scores": [],
        "f1_macro_scores": [],
        "f1_micro_scores": [],
        "mcc_scores": []
    }
    }

    accuracy_scores = []
    f1_macro_scores = []
    f1_micro_scores = []
    mcc_scores = []

    data_spike_exec_1_split_dict = {}
    for dataset in overall_result.keys():
        data_spike_exec_1_split_dict[dataset] = pd.DataFrame()

    session_id_for_df = session.get_trial_id()
    print(type(session_id_for_df))
    
    # save individual results for each dataset
    for i in range(5):
        eval_dict = evaluate_model_on_dataset_one_split(i, config)
        # fill all nan values in eval_dict with 0
        for dataset in eval_dict.keys():
            for key in eval_dict[dataset].keys():
                if np.isnan(eval_dict[dataset][key]):
                    eval_dict[dataset][key] = 0
        
        for dataset in overall_result.keys():

            overall_result[dataset]['accuracy_scores'].append(eval_dict[dataset]['accuracy'])
            overall_result[dataset]['f1_macro_scores'].append(eval_dict[dataset]['macro_f1'])
            overall_result[dataset]['f1_micro_scores'].append(eval_dict[dataset]['micro_f1'])
            overall_result[dataset]['mcc_scores'].append(eval_dict[dataset]['mcc'])

            accuracy_scores.append(eval_dict[dataset]['accuracy'])
            f1_macro_scores.append(eval_dict[dataset]['macro_f1'])
            f1_micro_scores.append(eval_dict[dataset]['micro_f1'])
            mcc_scores.append(eval_dict[dataset]['mcc'])
            
            new_entry= pd.DataFrame({
            #data_spike_exec_1_split_dict[dataset][dataset + "_" + str(i+1) + "_" + session_id_for_df] = {
                "name": [dataset + "_" + str(i+1) + "_" + session_id_for_df],
                "accuracy": [eval_dict[dataset]['accuracy']],
                "macro f1": [eval_dict[dataset]['macro_f1']],
                "micro_f1": [eval_dict[dataset]['micro_f1']],
                "mcc": [eval_dict[dataset]['mcc']],
                "config": [str(config)]
            })

            if i == 0:
                df_data_spike_1_split[dataset] = new_entry
            else:
                df_data_spike_1_split[dataset]= pd.concat([df_data_spike_1_split[dataset], new_entry], ignore_index=True, axis=0)

            


    for dataset in overall_result.keys():

        #data_spike_exec_full_split_dict[dataset][dataset + "_" + session_id_for_df] = {
        new_data_spike_exec_full_split_dict = pd.DataFrame({
                "name" : [dataset + "_" + session_id_for_df],
                "min_accuracy": [min(overall_result[dataset]['accuracy_scores'])],
                "max_accuracy": [max(overall_result[dataset]['accuracy_scores'])],
                "min_f1_macro": [min(overall_result[dataset]['f1_macro_scores'])],
                "max_f1_macro": [max(overall_result[dataset]['f1_macro_scores'])],
                "min_mcc": [min(overall_result[dataset]['mcc_scores'])],
                "max_mcc": [max(overall_result[dataset]['mcc_scores'])],
                "mean_accuracy": [np.mean(overall_result[dataset]['accuracy_scores'])],
                "mean_f1_macro": [np.mean(overall_result[dataset]['f1_macro_scores'])],
                "mean_f1_micro": [np.mean(overall_result[dataset]['f1_micro_scores'])],
                "mean_mcc": [np.mean(overall_result[dataset]['mcc_scores'])],
                "std_accuracy": [np.std(overall_result[dataset]['accuracy_scores'])],
                "std_f1_macro": [np.std(overall_result[dataset]['f1_macro_scores'])],
                "std_f1_micro": [np.std(overall_result[dataset]['f1_micro_scores'])],
                "std_mcc": [np.std(overall_result[dataset]['mcc_scores'])],
                "config": [str(config)]
        })

        full_split_path = os.path.join(helper.results_dir, f'Feature_based_output_full_{dataset}.pkl')
        if os.path.exists(full_split_path):
            df_existing_full = pd.read_pickle(full_split_path)
            df_data_spike_full_split = pd.concat([df_existing_full, new_data_spike_exec_full_split_dict], ignore_index=True, axis=0)
        else:
            print(f"No existing full split data found at {full_split_path}, creating new file.")
            df_data_spike_full_split = new_data_spike_exec_full_split_dict
        
        # Save the updated full split data to file
        df_data_spike_full_split.to_pickle(full_split_path)


        # Save the 1 split data using the full path
        split_path = os.path.join(helper.results_dir, f'Feature_based_output_{dataset}.pkl')
        if os.path.exists(split_path):
            df_existing_1_spike = pd.read_pickle(split_path)
            df_data_spike_1_split[dataset] = pd.concat([df_data_spike_1_split[dataset], df_existing_1_spike], ignore_index=True, axis=0)
        else:
            print(f"No existing 1 split data found at {split_path}, creating new file")

        # Save the updated 1 split data to file
        df_data_spike_1_split[dataset].to_pickle(split_path)

    if np.min(accuracy_scores) == 0:
        print(f"Zero accuracy detected in config: {config}")
        print(f"Accuracy scores: {accuracy_scores}")

    session.report({
        "min_mean_accuracy": np.min(accuracy_scores),
        "max_mean_accuracy": np.max(accuracy_scores),
        "mean_mean_accuracy": np.mean(accuracy_scores),
        "mean_mean_f1_macro": np.mean(f1_macro_scores),
        "mean_mean_f1_micro": np.mean(f1_micro_scores),
        "mean_mean_mcc": np.mean(mcc_scores)
    })

In [8]:
def generate_hidden_layers_config(min_layers=1, max_layers=5, min_nodes=10, max_nodes=50, step=10):
    possible_layers = []

    for num_layers in range(min_layers, max_layers + 1):

        shared_layers = list(itertools.product(range(min_nodes, max_nodes + 1, step), repeat=num_layers))

        max_output_layers = max_layers - num_layers
        
        output_layers = []
        for num_output_layers in range(0, max_output_layers + 1):

            output_layers = list(itertools.product(range(min_nodes, max_nodes + 1, step), repeat=num_output_layers))

        possible_layers.append({'shared': shared_layers, 'output': {'a2': output_layers, 'a3': output_layers, 'a4': output_layers, 'a12': output_layers, 'a21': output_layers}})

    return possible_layers

In [9]:
def five_fold_cross_validation(num_layers):

    num_layers = num_layers - 1

    global best_params_list_getting

    hidden_layers_options = generate_hidden_layers_config()

    config = {
        "activation": tune.choice(["relu", "tanh", "sigmoid"]),
        "learning_rate": tune.loguniform(1e-4, 1e-2),
        "batch_size": tune.choice([32, 64, 128]),
        "shared_hidden_layers": tune.choice(hidden_layers_options[num_layers]['shared']),
        "a2_output_hidden_layers": tune.choice(hidden_layers_options[num_layers]['output']['a2']),
        "a3_output_hidden_layers": tune.choice(hidden_layers_options[num_layers]['output']['a3']),
        "a4_output_hidden_layers": tune.choice(hidden_layers_options[num_layers]['output']['a4']),
        "a12_output_hidden_layers": tune.choice(hidden_layers_options[num_layers]['output']['a12']),
        "a21_output_hidden_layers": tune.choice(hidden_layers_options[num_layers]['output']['a21']),
        "epochs": tune.choice([10, 20, 30, 40, 50]),
        "dropout_rate": tune.uniform(0.2, 0.5)
    }
    
    scheduler = ASHAScheduler(
        metric="mean_mean_accuracy",
        mode="max",
        max_t=10,
        grace_period=1,
        reduction_factor=2
    )
    
    search_alg = OptunaSearch(metric="mean_mean_accuracy", mode="max")
    
    analysis = tune.run(
        tune.with_parameters(train_and_evaluate),
        resources_per_trial={"cpu": 10, "gpu": 0, "accelerator_type:RTX": 0},
        config=config,
        scheduler=scheduler,
        search_alg=search_alg,
        num_samples=10,
        verbose=1,
        storage_path=helper.ray_results_dir,
        trial_dirname_creator=custom_trial_dirname
    )

    best_config_data_ray_tune = analysis.get_best_config(metric="mean_mean_accuracy", mode="max")
    print("Best hyperparameters found were: ", best_config_data_ray_tune)
    best_params_list_getting.append(best_config_data_ray_tune)
    
    return analysis

In [10]:
analysis = five_fold_cross_validation(5)

(train_and_evaluate pid=33275) <class 'str'>


2024-08-08 10:36:25,275	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (10, 20, 50, 20, 30), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': ()}


(train_and_evaluate pid=33275) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/Feature_based_output_full_a2.pkl, creating new file.
(train_and_evaluate pid=33275) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/Feature_based_output_a2.pkl, creating new file
(train_and_evaluate pid=33275) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/Feature_based_output_full_a3.pkl, creating new file.
(train_and_evaluate pid=33275) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/Feature_based_output_a3.pkl, creating new file
(train_and_evaluate pid=33275) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/Feature_based_output_full_a4.pkl, creating new file

2024-08-08 10:37:03,635	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (40, 50, 30, 20, 50), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': ()}


(train_and_evaluate pid=33511) <class 'str'>


...

(train_and_evaluate pid=34781) <class 'str'>


2024-08-08 10:49:23,076	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (40, 20, 20, 10, 50), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': ()}


(train_and_evaluate pid=34976) <class 'str'>


2024-08-08 10:51:10,773	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (30, 10, 10, 30, 10), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': ()}
2024-08-08 10:51:10,780	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/ray_results/train_and_evaluate_2024-08-08_10-35-00' in 0.0064s.
2024-08-08 10:51:10,784	INFO tune.py:1041 -- Total run time: 970.08 seconds (970.00 seconds for the tuning loop).


Best hyperparameters found were:  {'activation': 'tanh', 'learning_rate': 0.0009012544512547965, 'batch_size': 32, 'shared_hidden_layers': (10, 10, 50, 30, 20), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': (), 'epochs': 50, 'dropout_rate': 0.4271563988934437}


In [11]:
analysis = five_fold_cross_validation(4)

(train_and_evaluate pid=35147) <class 'str'>


2024-08-08 10:51:33,662	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (40, 10, 20, 10), 'a2_output_hidden_layers': (20,), 'a3_output_hidden_layers': (10,), 'a4_output_hidden_layers': (50,), 'a12_output_hidden_layers': (50,), 'a21_output_hidden_layers': (30,)}


(train_and_evaluate pid=35193) <class 'str'>


2024-08-08 10:52:48,180	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (10, 10, 40, 50), 'a2_output_hidden_layers': (20,), 'a3_output_hidden_layers': (30,), 'a4_output_hidden_layers': (20,), 'a12_output_hidden_layers': (50,), 'a21_output_hidden_layers': (50,)}


(train_and_evaluate pid=35333) <class 'str'>


...

(train_and_evaluate pid=36299) <class 'str'>


2024-08-08 11:02:27,725	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (20, 20, 20, 30), 'a2_output_hidden_layers': (20,), 'a3_output_hidden_layers': (30,), 'a4_output_hidden_layers': (50,), 'a12_output_hidden_layers': (40,), 'a21_output_hidden_layers': (50,)}


(train_and_evaluate pid=36461) <class 'str'>


2024-08-08 11:03:14,548	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (50, 40, 20, 20), 'a2_output_hidden_layers': (40,), 'a3_output_hidden_layers': (30,), 'a4_output_hidden_layers': (20,), 'a12_output_hidden_layers': (20,), 'a21_output_hidden_layers': (20,)}
2024-08-08 11:03:14,557	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/ray_results/train_and_evaluate_2024-08-08_10-51-10' in 0.0071s.
2024-08-08 11:03:14,562	INFO tune.py:1041 -- Total run time: 723.74 seconds (723.70 seconds for the tuning loop).


Best hyperparameters found were:  {'activation': 'tanh', 'learning_rate': 0.001137191002648876, 'batch_size': 128, 'shared_hidden_layers': (50, 40, 20, 20), 'a2_output_hidden_layers': (40,), 'a3_output_hidden_layers': (30,), 'a4_output_hidden_layers': (20,), 'a12_output_hidden_layers': (20,), 'a21_output_hidden_layers': (20,), 'epochs': 50, 'dropout_rate': 0.3383933955559252}


In [12]:
analysis = five_fold_cross_validation(3)

(train_and_evaluate pid=36532) <class 'str'>


2024-08-08 11:05:53,351	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (20, 20, 50), 'a2_output_hidden_layers': (50, 50), 'a3_output_hidden_layers': (10, 40), 'a4_output_hidden_layers': (10, 50), 'a12_output_hidden_layers': (20, 20), 'a21_output_hidden_layers': (10, 30)}


(train_and_evaluate pid=36825) <class 'str'>


2024-08-08 11:08:09,164	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (40, 10, 30), 'a2_output_hidden_layers': (10, 40), 'a3_output_hidden_layers': (20, 30), 'a4_output_hidden_layers': (50, 40), 'a12_output_hidden_layers': (50, 30), 'a21_output_hidden_layers': (50, 30)}


(train_and_evaluate pid=37091) <class 'str'>


...

(train_and_evaluate pid=38024) <class 'str'>


2024-08-08 11:16:09,136	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (50, 20, 20), 'a2_output_hidden_layers': (10, 40), 'a3_output_hidden_layers': (40, 10), 'a4_output_hidden_layers': (10, 40), 'a12_output_hidden_layers': (10, 50), 'a21_output_hidden_layers': (10, 10)}


(train_and_evaluate pid=38103) <class 'str'>


2024-08-08 11:18:53,257	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (50, 50, 50), 'a2_output_hidden_layers': (20, 10), 'a3_output_hidden_layers': (50, 50), 'a4_output_hidden_layers': (10, 40), 'a12_output_hidden_layers': (40, 30), 'a21_output_hidden_layers': (30, 20)}
2024-08-08 11:18:53,265	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/ray_results/train_and_evaluate_2024-08-08_11-03-14' in 0.0062s.
2024-08-08 11:18:53,270	INFO tune.py:1041 -- Total run time: 938.69 seconds (938.64 seconds for the tuning loop).


Best hyperparameters found were:  {'activation': 'tanh', 'learning_rate': 0.0001558560745128232, 'batch_size': 32, 'shared_hidden_layers': (20, 20, 50), 'a2_output_hidden_layers': (50, 50), 'a3_output_hidden_layers': (10, 40), 'a4_output_hidden_layers': (10, 50), 'a12_output_hidden_layers': (20, 20), 'a21_output_hidden_layers': (10, 30), 'epochs': 50, 'dropout_rate': 0.3124471333566296}


In [13]:
analysis = five_fold_cross_validation(2)

(train_and_evaluate pid=38349) <class 'str'>


2024-08-08 11:19:14,358	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (30, 50), 'a2_output_hidden_layers': (40, 10, 30), 'a3_output_hidden_layers': (30, 30, 40), 'a4_output_hidden_layers': (50, 40, 10), 'a12_output_hidden_layers': (50, 10, 10), 'a21_output_hidden_layers': (10, 40, 40)}


(train_and_evaluate pid=38384) <class 'str'>


2024-08-08 11:20:34,148	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (40, 40), 'a2_output_hidden_layers': (40, 30, 30), 'a3_output_hidden_layers': (30, 30, 30), 'a4_output_hidden_layers': (20, 50, 40), 'a12_output_hidden_layers': (20, 40, 50), 'a21_output_hidden_layers': (20, 10, 20)}


(train_and_evaluate pid=38506) <class 'str'>


...

(train_and_evaluate pid=38962) <class 'str'>


2024-08-08 11:26:52,143	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (20, 50), 'a2_output_hidden_layers': (50, 50, 10), 'a3_output_hidden_layers': (10, 20, 50), 'a4_output_hidden_layers': (20, 10, 20), 'a12_output_hidden_layers': (50, 50, 40), 'a21_output_hidden_layers': (20, 30, 40)}


(train_and_evaluate pid=39083) <class 'str'>


2024-08-08 11:27:55,817	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (30, 20), 'a2_output_hidden_layers': (30, 20, 20), 'a3_output_hidden_layers': (20, 10, 40), 'a4_output_hidden_layers': (10, 10, 10), 'a12_output_hidden_layers': (30, 50, 30), 'a21_output_hidden_layers': (40, 30, 30)}
2024-08-08 11:27:55,826	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/ray_results/train_and_evaluate_2024-08-08_11-18-53' in 0.0069s.
2024-08-08 11:27:55,831	INFO tune.py:1041 -- Total run time: 542.54 seconds (542.50 seconds for the tuning loop).


Best hyperparameters found were:  {'activation': 'tanh', 'learning_rate': 0.0035966985028063796, 'batch_size': 64, 'shared_hidden_layers': (40, 40), 'a2_output_hidden_layers': (40, 30, 30), 'a3_output_hidden_layers': (30, 30, 30), 'a4_output_hidden_layers': (20, 50, 40), 'a12_output_hidden_layers': (20, 40, 50), 'a21_output_hidden_layers': (20, 10, 20), 'epochs': 50, 'dropout_rate': 0.37364854091239985}


In [14]:
analysis = five_fold_cross_validation(1)

(train_and_evaluate pid=39178) <class 'str'>


2024-08-08 11:29:25,965	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (30,), 'a2_output_hidden_layers': (30, 10, 10, 30), 'a3_output_hidden_layers': (50, 10, 10, 50), 'a4_output_hidden_layers': (20, 20, 20, 40), 'a12_output_hidden_layers': (30, 40, 50, 20), 'a21_output_hidden_layers': (20, 50, 40, 40)}


(train_and_evaluate pid=39316) <class 'str'>


2024-08-08 11:29:46,779	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (40,), 'a2_output_hidden_layers': (50, 10, 10, 20), 'a3_output_hidden_layers': (40, 10, 20, 50), 'a4_output_hidden_layers': (10, 40, 10, 40), 'a12_output_hidden_layers': (30, 40, 10, 30), 'a21_output_hidden_layers': (40, 40, 40, 40)}


(train_and_evaluate pid=39349) <class 'str'>


...

(train_and_evaluate pid=39873) <class 'str'>


2024-08-08 11:36:11,809	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (30,), 'a2_output_hidden_layers': (30, 20, 20, 40), 'a3_output_hidden_layers': (50, 30, 30, 30), 'a4_output_hidden_layers': (20, 30, 50, 10), 'a12_output_hidden_layers': (30, 10, 40, 10), 'a21_output_hidden_layers': (50, 50, 30, 30)}


(train_and_evaluate pid=39942) <class 'str'>


2024-08-08 11:37:46,684	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (30,), 'a2_output_hidden_layers': (10, 50, 20, 10), 'a3_output_hidden_layers': (40, 50, 20, 20), 'a4_output_hidden_layers': (50, 40, 50, 20), 'a12_output_hidden_layers': (20, 10, 50, 30), 'a21_output_hidden_layers': (40, 10, 30, 30)}
2024-08-08 11:37:46,694	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/ray_results/train_and_evaluate_2024-08-08_11-27-55' in 0.0083s.
2024-08-08 11:37:46,698	INFO tune.py:1041 -- Total run time: 590.86 seconds (590.81 seconds for the tuning loop).


Best hyperparameters found were:  {'activation': 'tanh', 'learning_rate': 0.0004091349628370055, 'batch_size': 128, 'shared_hidden_layers': (30,), 'a2_output_hidden_layers': (40, 50, 30, 30), 'a3_output_hidden_layers': (50, 10, 50, 30), 'a4_output_hidden_layers': (40, 20, 50, 20), 'a12_output_hidden_layers': (30, 10, 40, 50), 'a21_output_hidden_layers': (40, 30, 20, 20), 'epochs': 50, 'dropout_rate': 0.49371946100111525}


In [15]:
analysis = five_fold_cross_validation(0)

(train_and_evaluate pid=40087) <class 'str'>


2024-08-08 11:39:06,767	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (40, 40, 20, 40, 20), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': ()}


(train_and_evaluate pid=40209) <class 'str'>


2024-08-08 11:39:36,529	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (20, 30, 10, 40, 10), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': ()}


(train_and_evaluate pid=40254) <class 'str'>


...

(train_and_evaluate pid=40860) <class 'str'>


2024-08-08 11:46:21,951	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (20, 40, 20, 20, 50), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': ()}


(train_and_evaluate pid=40881) <class 'str'>


2024-08-08 11:46:58,970	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'shared_hidden_layers': (10, 40, 20, 30, 10), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': ()}
2024-08-08 11:46:58,981	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/ray_results/train_and_evaluate_2024-08-08_11-37-46' in 0.0086s.
2024-08-08 11:46:58,989	INFO tune.py:1041 -- Total run time: 552.28 seconds (552.21 seconds for the tuning loop).


Best hyperparameters found were:  {'activation': 'relu', 'learning_rate': 0.0006360430333978182, 'batch_size': 64, 'shared_hidden_layers': (40, 40, 20, 40, 20), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': (), 'epochs': 50, 'dropout_rate': 0.3246924305272304}


In [16]:
df_dict_A2 = pd.read_pickle("results/Feature_based_output_A2.pkl")
df_dict_A3 = pd.read_pickle("results/Feature_based_output_A3.pkl")
df_dict_A4 = pd.read_pickle("results/Feature_based_output_A4.pkl")
df_dict_A12 = pd.read_pickle("results/Feature_based_output_A12.pkl")
df_dict_A21 = pd.read_pickle("results/Feature_based_output_A21.pkl")

df_dict = pd.concat([df_dict_A2, df_dict_A3, df_dict_A4, df_dict_A12, df_dict_A21])

df_dict = df_dict.loc[:,~df_dict.columns.duplicated()].copy()
df_dict = df_dict.drop_duplicates()

df_dict.to_csv('outputs/Feature_based_one_split_metrics_system.csv')

In [17]:
df_dict_full_A2 = pd.read_pickle("results/Feature_based_output_full_A2.pkl")
df_dict_full_A3 = pd.read_pickle("results/Feature_based_output_full_A3.pkl")
df_dict_full_A4 = pd.read_pickle("results/Feature_based_output_full_A4.pkl")
df_dict_full_A12 = pd.read_pickle("results/Feature_based_output_full_A12.pkl")
df_dict_full_A21 = pd.read_pickle("results/Feature_based_output_full_A21.pkl")

df_full_dict = pd.concat([df_dict_full_A2, df_dict_full_A3, df_dict_full_A4, df_dict_full_A12, df_dict_full_A21])

df_full_dict = df_full_dict.loc[:, ~df_full_dict.columns.duplicated()].copy()
df_full_dict = df_full_dict.drop_duplicates()

df_full_dict.to_csv("outputs/Feature_based__full_split_metrics_system.csv")

In [18]:
df_dict_full_A2.to_csv("outputs/Feature_based_full_split_metrics_A2.csv")
df_dict_full_A3.to_csv("outputs/Feature_based_full_split_metrics_A3.csv")
df_dict_full_A4.to_csv("outputs/Feature_based_full_split_metrics_A4.csv")
df_dict_full_A12.to_csv("outputs/Feature_based_full_split_metrics_A12.csv")
df_dict_full_A21.to_csv("outputs/Feature_based_full_split_metrics_A21.csv")

In [19]:
# print the configs and results for the hyperparemters wiht the highest mean accuracy
# read df_full_dict and print columns with configs in best_params_list_getting
df_full_dict = pd.read_csv("outputs/Feature_based__full_split_metrics_system.csv")

config_acc_dict = {}
for good_param in best_params_list_getting:
    # get the row witht the highest mean mean accuracy
    mean_accuracy = df_full_dict.loc[df_full_dict['config'] == str(good_param)]['mean_accuracy'].mean()
    config_acc_dict[str(good_param)] = mean_accuracy
    print(f"Mean accuracy for config {good_param} is {mean_accuracy}")

# get the best config
best_config = max(config_acc_dict, key=config_acc_dict.get)
print(f"The best config is {best_config} with a mean accuracy of {config_acc_dict[best_config]}")

#print aggregate results for the best config
metrics = ['name', 'mean_accuracy', 'std_accuracy', 'mean_f1_macro', 'std_f1_macro', 'mean_f1_micro', 'std_f1_micro', 'mean_mcc', 'std_mcc']
print(df_full_dict[metrics].loc[df_full_dict['config'] == best_config])

Mean accuracy for config {'activation': 'tanh', 'learning_rate': 0.0009012544512547965, 'batch_size': 32, 'shared_hidden_layers': (10, 10, 50, 30, 20), 'a2_output_hidden_layers': (), 'a3_output_hidden_layers': (), 'a4_output_hidden_layers': (), 'a12_output_hidden_layers': (), 'a21_output_hidden_layers': (), 'epochs': 50, 'dropout_rate': 0.4271563988934437} is 0.6107975608816163
Mean accuracy for config {'activation': 'tanh', 'learning_rate': 0.001137191002648876, 'batch_size': 128, 'shared_hidden_layers': (50, 40, 20, 20), 'a2_output_hidden_layers': (40,), 'a3_output_hidden_layers': (30,), 'a4_output_hidden_layers': (20,), 'a12_output_hidden_layers': (20,), 'a21_output_hidden_layers': (20,), 'epochs': 50, 'dropout_rate': 0.3383933955559252} is 0.6122718885721694
Mean accuracy for config {'activation': 'tanh', 'learning_rate': 0.0001558560745128232, 'batch_size': 32, 'shared_hidden_layers': (20, 20, 50), 'a2_output_hidden_layers': (50, 50), 'a3_output_hidden_layers': (10, 40), 'a4_outpu